# Overview

Create 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import os

# Create useable weight series

In [46]:
'''
Assumes weight at end_time is 1 and weight at start_time is 1/2
Input:
    index_value       integer
    start_time        integer
    end_time          integer
Return:
    float
'''
def CalculateTimeWeight(index_value, start_time, end_time):
    return np.exp(np.log(0.5)/(float(start_time - end_time))*(index_value-end_time))

In [67]:
aaa = pd.read_pickle('../Data/MolecularFoundryPickleFiles_3-cleaned-units-converted-engfeatures/ARU1_max75min1.p')

In [54]:
weights = pd.Series(index=aaa.index)
for index in weights.index:
    weights[index] = CalculateTimeWeight(index, weights.index.min(), weights.index.max())

In [56]:
weights.to_pickle('timeweights.p')

# Class Labels

In the terminal, copy the /labels folder from step 2 or so

In [3]:
# Get list of files
filenames = os.listdir('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv/labels')

In [4]:
aaa = pd.read_pickle('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv/labels/ARU1_max75min1-label.p')

In [61]:
filenames

['ARU1_max75min1-label.p',
 'ARU2_max75min1-label.p',
 'ARU3_max75min1-label.p',
 'coolingtower1_efficiency75percent-label.p',
 'coolingtower2_efficiency75percent-label.p',
 'GP1_efficiencyx75percent-label.p',
 'GP20_efficiencyx75percent-label.p',
 'GP21_efficiencyx75percent-label.p',
 'GP2_efficiencyx75percent-label.p',
 'GP4_efficiencyx75percent-label.p',
 'GP5_efficiencyx75percent-label.p',
 'GP6_efficiencyx75percent-label.p',
 'no_faults-label.p']

# Create Data Sets for Time Series Cross Validation

In [9]:
aaa = pd.read_pickle('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv/ARU1_max75min1.p')

In [5]:
# Create the output directory
if not os.path.exists('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv'):
    os.makedirs('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv')

In [4]:
def weighByClass(temp_train):
    # Calculate the class weight
    num_in_class = np.bincount(temp_train['__label__'])
    for class_i in range(0, len(num_in_class)):
        if num_in_class[class_i] == 0:
            num_in_class[class_i] = 1
    num_samples = len(temp_train)
    num_classes = 2
    classweight = {}
    classweight[0] = num_samples/float(num_classes * num_in_class[0])
    classweight[1] = num_samples/float(num_classes * num_in_class[1])    
    temp_train.loc[temp_train['__label__']==0, '__weight__'] = temp_train[temp_train['__label__']==0]['__weight__'] * classweight[0]
    temp_train.loc[temp_train['__label__']==1, '__weight__'] = temp_train[temp_train['__label__']==1]['__weight__'] * classweight[1]


In [5]:
# Get list of pickle files
all_filenames = os.listdir('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv/')
filenames = []
for filename in all_filenames:
    if '.p' in filename:
        filenames.append(filename)
        print(filename)
del all_filenames

ARU1_max75min1.p
GP20_efficiencyx75percent.p
ARU2_max75min1.p
ARU3_max75min1.p
coolingtower1_efficiency75percent.p
coolingtower2_efficiency75percent.p
GP21_efficiencyx75percent.p
GP1_efficiencyx75percent.p
GP4_efficiencyx75percent.p
no_faults.p
GP6_efficiencyx75percent.p
GP2_efficiencyx75percent.p
GP5_efficiencyx75percent.p


In [22]:
num_folds = 12 # 12 months
# Concatenate the labels and time weights
length_of_one_fold = int(np.floor(len(aaa.index)/(num_folds)))
# Find the boundaries of the folds
fold_boundaries = []
for fold_i in range(0, num_folds): #subtract one because the last one is an unequal number
    fold_boundaries.append(length_of_one_fold*fold_i)
fold_boundaries.append(len(aaa.index))

In [23]:
for fold_i in range(0, len(fold_boundaries)):
    print('Fold %d: %d' % (fold_i, fold_boundaries[fold_i]))

Fold 0: 0
Fold 1: 8759
Fold 2: 17518
Fold 3: 26277
Fold 4: 35036
Fold 5: 43795
Fold 6: 52554
Fold 7: 61313
Fold 8: 70072
Fold 9: 78831
Fold 10: 87590
Fold 11: 96349
Fold 12: 105109


In [ ]:
timeweights = pd.read_pickle('timeweights.p')
#for fold_i in range(0, len(fold_boundaries)-2):
for fold_i in range(0, 7):
    training_end_index = fold_boundaries[fold_i+1]
    testing_start_index = fold_boundaries[fold_i+1]
    testing_end_index = fold_boundaries[fold_i+2]
    
    print('Fold %d to %d' % (fold_i, training_end_index))
    
    training_data = pd.DataFrame()
    testing_data = pd.DataFrame()
    
    for filename in filenames:

        #print('traing to %d' %training_end_index)
        #print('%d to %d' % (testing_start_index, testing_end_index))

        # Load all X Data
        print('Loading %s' % filename)
        filepath = os.path.join('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv', filename)
        df = pd.read_pickle(filepath)

        # Load all Y Data
        filename = os.path.splitext(filename)[0] + '-label.p'
        print('Loading %s' % filename)
        filepath = os.path.join('../Data/MolecularFoundryPickleFiles_4-cleaned-units-converted-engfeatures-lagstdv/labels/', filename)
        labels = pd.read_pickle(filepath)

        # Get the fold
        temp_train = df.iloc[0:training_end_index, :].copy()
        temp_train['__label__'] = labels[temp_train.index]

        # Time weight
        temp_train['__weight__'] = timeweights[temp_train.index+(df.index.max() - temp_train.index.max())].values # Make 1 month old data weights as 1 month old, regardless of when the month is


        temp_test = df.iloc[testing_start_index:testing_end_index, :].copy()
        temp_test['__label__'] = labels[temp_test.index]
        
        del df

        training_data = training_data.append(temp_train)
        testing_data = testing_data.append(temp_test)
        del temp_train
        del temp_test

    weighByClass(training_data)  
    training_data.to_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/training_data_cv' + str(fold_i+1) + '.p')
    del training_data
    testing_data.to_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/testing_data_cv' + str(fold_i+1)+ '.p')
    del testing_data

# Calculate Mean and Std

In [13]:
aaa = pd.read_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/training_data_cv11.p')
stds = aaa.std()
means = aaa.mean()
stds = stds.drop(['__label__', '__weight__'])
stds.to_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/stds.p')
means = means.drop(['__label__', '__weight__'])
means.to_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/means.p')

# Whiten the data

In [15]:
# Get list of pickle files
all_filenames = os.listdir('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv')
filenames = []
for filename in all_filenames:
    if ('.p' in filename) and ('data' in filename):
        filenames.append(filename)
        print(filename)
del all_filenames

training_data_cv10.p
testing_data_cv10.p
training_data_cv11.p
testing_data_cv11.p
training_data_cv8.p
testing_data_cv8.p


In [19]:
stds = pd.read_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/stds.p')
means = pd.read_pickle('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv/means.p')
zerostd_var = stds[stds==0].index.values
means = means.drop(zerostd_var)
stds = stds.drop(zerostd_var)
means = means.drop(stds[stds==0].index.values)

In [20]:
for filename in filenames:
    print('Running %s ' % filename)
    # Remove zero variance features
    filepath_read = os.path.join('../Data/MolecularFoundryPickleFiles_5-cleaned-units-engfeatures-lagstdv-cv', filename)
    df = pd.read_pickle(filepath_read)
    df = df.drop(zerostd_var, 1)
    
    # Subtract the mean
    for feature_i in means.index:
        df.loc[:, feature_i] = (df.loc[:, feature_i] - means[feature_i])

    # Divide by the standard deviation
    for feature_i in stds.index:
        df.loc[:, feature_i] = (df.loc[:, feature_i])  / stds[feature_i]
    
    # Save the pickle file
    filepath_write = os.path.join('../Data/MolecularFoundryPickleFiles_6-cleaned-units-engfeatures-lagstdv-cv-nonzerovar', filename)
    df.to_pickle(filepath_write)
    print('Done %s ' % filename)

Running training_data_cv10.p 
Done training_data_cv10.p 
Running testing_data_cv10.p 
Done testing_data_cv10.p 
Running training_data_cv11.p 
Done training_data_cv11.p 
Running testing_data_cv11.p 
Done testing_data_cv11.p 
Running training_data_cv8.p 
Done training_data_cv8.p 
Running testing_data_cv8.p 
Done testing_data_cv8.p 
